In [1]:
import statsmodels.formula.api as smf
import fastreg as fr
from fastreg import I, R, C, C0

### Generate Data

In [2]:
models = ['linear', 'poisson', 'logit', 'negbin']
data = fr.dataset(N=1_000_000, K1=10, K2=100, models=models, seed=89320432)
data_wide = fr.dataset(N=1_000_000, K1=10, K2=10_000, models=models, seed=89320433)
data.head()

,x1,x2,yhat0,yhat,id1,id2,y0,y,Eb0,Eb,b0,b,Ep0,Ep,p0,p,nb0,nb
0,-1.429225,1.830295,0.769410,2.149410,H,68,1.791671,1.585510,0.683393,0.895614,1,1,2.158492,8.579792,4,10,1,5
1,0.687153,0.795373,0.783369,1.403369,C,42,0.295552,3.163529,0.686406,0.802718,1,1,2.188835,4.068886,1,2,1,0
2,0.764353,-1.060225,-0.306829,-0.036829,C,7,0.381637,-0.639362,0.423889,0.490794,1,1,0.735777,0.963841,0,1,2,0
3,-0.309887,-0.777701,-0.459587,0.800413,D,96,0.143218,1.139854,0.387084,0.690063,0,1,0.631545,2.226461,0,0,1,3
4,-0.616042,-0.752266,-0.536172,0.623828,H,46,-0.468395,1.462211,0.369078,0.651089,1,1,0.584983,1.866057,0,1,1,7


In [3]:
# for statsmodels runs
data1 = data.copy()
data1['id2'] = data1['id2'].astype(str)

### Normal OLS

In [4]:
%time fr.ols(y=R.y0, x=I+R.x1+R.x2, data=data)

CPU times: user 53 ms, sys: 93.2 ms, total: 146 ms
Wall time: 22.7 ms


y0,coeff,stderr,low95,high95,pvalue
I,0.101859,0.001000,0.099900,0.103818,0.0
x1,0.301647,0.001000,0.299688,0.303606,0.0
x2,0.599408,0.000999,0.597450,0.601366,0.0


In [5]:
%time smf.ols('y0 ~ 1 + x1 + x2', data=data).fit().params

CPU times: user 363 ms, sys: 1.24 s, total: 1.6 s
Wall time: 134 ms


Intercept    0.101859
x1           0.301647
x2           0.599408
dtype: float64

In [6]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

CPU times: user 445 ms, sys: 796 ms, total: 1.24 s
Wall time: 321 ms


y,coeff,stderr,low95,high95,pvalue
I,0.112463,0.010413,0.092054,0.132872,0.0
x1,0.299557,0.001001,0.297595,0.301518,0.0
x2,0.601840,0.001000,0.599880,0.603800,0.0
id1=B,0.104734,0.004476,0.095962,0.113507,0.0
id1=C,0.201965,0.004474,0.193197,0.210734,0.0
...,...,...,...,...,...
id2=95,0.937461,0.014102,0.909821,0.965101,0.0
id2=96,0.956624,0.014109,0.928970,0.984278,0.0
id2=97,0.960951,0.014099,0.933318,0.988584,0.0
id2=98,0.993398,0.014132,0.965699,1.021097,0.0


In [7]:
%time smf.ols('y ~ 1 + x1 + x2 + id1 + id2', data=data1).fit().params

CPU times: user 1min 39s, sys: 14.6 s, total: 1min 53s
Wall time: 10.7 s


Intercept    0.112463
id1[T.B]     0.104734
id1[T.C]     0.201965
id1[T.D]     0.300212
id1[T.E]     0.403094
               ...   
id2[T.97]    0.960951
id2[T.98]    0.993398
id2[T.99]    0.988670
x1           0.299557
x2           0.601840
Length: 111, dtype: float64

### High Dimensional

In [8]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1+C.id2, data=data_wide)

CPU times: user 3.99 s, sys: 2.41 s, total: 6.4 s
Wall time: 4.42 s


y,coeff,stderr,low95,high95,pvalue
I,0.143877,0.103699,-0.059369,0.347123,1.653041e-01
x1,0.301630,0.001004,0.299662,0.303597,0.000000e+00
x2,0.599284,0.001006,0.597313,0.601255,0.000000e+00
id1=B,0.097642,0.004499,0.088824,0.106461,0.000000e+00
id1=C,0.189008,0.004491,0.180207,0.197810,0.000000e+00
...,...,...,...,...,...
id2=9995,1.045182,0.146196,0.758643,1.331722,8.730794e-13
id2=9996,1.029893,0.146587,0.742588,1.317198,2.128075e-12
id2=9997,0.892911,0.142021,0.614554,1.171267,3.233234e-10
id2=9998,0.961372,0.144705,0.677755,1.244989,3.059886e-11


In [9]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1, hdfe=C.id2, data=data_wide)

CPU times: user 368 ms, sys: 41.1 ms, total: 409 ms
Wall time: 358 ms


y,coeff,stderr,low95,high95,pvalue
I,0.143877,0.103699,-0.059369,0.347123,1.653041e-01
x1,0.301630,0.001004,0.299662,0.303597,0.000000e+00
x2,0.599284,0.001006,0.597313,0.601255,0.000000e+00
id1=B,0.097642,0.004499,0.088824,0.106461,0.000000e+00
id1=C,0.189008,0.004491,0.180207,0.197810,0.000000e+00
...,...,...,...,...,...
id2=9995,1.045182,0.146196,0.758643,1.331722,8.730794e-13
id2=9996,1.029893,0.146587,0.742588,1.317198,2.128075e-12
id2=9997,0.892911,0.142021,0.614554,1.171267,3.233234e-10
id2=9998,0.961372,0.144705,0.677755,1.244989,3.059886e-11


In [10]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1, absorb=C.id2, data=data_wide)

CPU times: user 781 ms, sys: 1.87 s, total: 2.65 s
Wall time: 352 ms


y,coeff,stderr,low95,high95,pvalue
I,0.605524,0.003035,0.599576,0.611471,0.0
x1,0.301630,0.000998,0.299674,0.303585,0.0
x2,0.599284,0.001014,0.597297,0.601270,0.0
id1=B,0.097642,0.004531,0.088762,0.106523,0.0
id1=C,0.189008,0.004545,0.180100,0.197917,0.0
id1=D,0.295160,0.004506,0.286328,0.303991,0.0
id1=E,0.402802,0.004510,0.393963,0.411641,0.0
id1=F,0.495409,0.004493,0.486602,0.504215,0.0
id1=G,0.596910,0.004467,0.588155,0.605665,0.0
id1=H,0.694326,0.004490,0.685526,0.703125,0.0


### Poisson

In [11]:
%time fr.poisson(y=R.p0, x=I+R.x1+R.x2, data=data)

[  0] ℓ=-0.63746, g=0.02879, Δβ=0.59645, Δℓ=inf, μR=0.33240, μC=nan
[ 25] ℓ=-0.62218, g=0.00001, Δβ=0.00010, Δℓ=0.00000, μR=0.33596, μC=nan
CPU times: user 6.95 s, sys: 1.76 s, total: 8.71 s
Wall time: 5.78 s


p0,coeff,stderr,low95,high95,pvalue
I,0.098597,0.001023,0.096592,0.100601,0.0
x1,0.306138,0.000849,0.304474,0.307801,0.0
x2,0.603147,0.000847,0.601487,0.604807,0.0


In [12]:
%time fr.poisson(y=R.p, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

[  0] ℓ=2.39633, g=0.10135, Δβ=0.81832, Δℓ=inf, μR=0.43051, μC=0.34926
[ 40] ℓ=2.55442, g=0.00002, Δβ=0.00087, Δℓ=0.00000, μR=0.33897, μC=0.49472
[ 60] ℓ=2.55441, g=0.00000, Δβ=0.00010, Δℓ=0.00000, μR=0.33669, μC=0.49806
CPU times: user 25.4 s, sys: 5.76 s, total: 31.2 s
Wall time: 19 s


p,coeff,stderr,low95,high95,pvalue
I,0.105451,0.006940,0.091848,0.119054,0.0
x1,0.299632,0.000507,0.298637,0.300626,0.0
x2,0.604986,0.000505,0.603996,0.605976,0.0
id1=B,0.097879,0.002831,0.092330,0.103428,0.0
id1=C,0.192535,0.002770,0.187107,0.197964,0.0
...,...,...,...,...,...
id2=95,0.957098,0.007779,0.941851,0.972345,0.0
id2=96,0.932674,0.007809,0.917368,0.947980,0.0
id2=97,0.971570,0.007815,0.956252,0.986887,0.0
id2=98,1.001781,0.007772,0.986548,1.017014,0.0


### Logit

In [13]:
%time fr.logit(y=R.b0, x=I+R.x1+R.x2, data=data)

[  0] ℓ=-0.64533, g=0.00326, Δβ=0.60308, Δℓ=inf, μR=0.33762, μC=nan
[ 16] ℓ=-0.64339, g=0.00000, Δβ=0.00009, Δℓ=0.00000, μR=0.34224, μC=nan
CPU times: user 4.5 s, sys: 991 ms, total: 5.49 s
Wall time: 3.29 s


b0,coeff,stderr,low95,high95,pvalue
I,0.109110,0.002104,0.104986,0.113234,0.0
x1,0.309857,0.002146,0.305651,0.314062,0.0
x2,0.607748,0.002269,0.603300,0.612195,0.0


In [14]:
%time fr.logit(y=R.b, x=I+R.x1+R.x2+C.id1+C.id2, data=data)

[  0] ℓ=-0.54980, g=0.00198, Δβ=0.81912, Δℓ=inf, μR=0.42639, μC=0.35674
[ 40] ℓ=-0.54397, g=0.00001, Δβ=0.00124, Δℓ=0.00000, μR=0.35642, μC=0.46521
[ 80] ℓ=-0.54397, g=0.00000, Δβ=0.00018, Δℓ=0.00000, μR=0.34942, μC=0.47594
[ 92] ℓ=-0.54397, g=0.00000, Δβ=0.00010, Δℓ=0.00000, μR=0.34890, μC=0.47673
CPU times: user 35.8 s, sys: 6.99 s, total: 42.8 s
Wall time: 26.5 s


b,coeff,stderr,low95,high95,pvalue
I,0.154282,0.022841,0.109513,0.199050,1.433609e-11
x1,0.296371,0.002381,0.291704,0.301039,0.000000e+00
x2,0.596045,0.002508,0.591130,0.600960,0.000000e+00
id1=B,0.079692,0.009834,0.060417,0.098966,4.440892e-16
id1=C,0.204975,0.009909,0.185554,0.224397,0.000000e+00
...,...,...,...,...,...
id2=95,0.832166,0.033493,0.766521,0.897812,0.000000e+00
id2=96,0.946437,0.034055,0.879690,1.013184,0.000000e+00
id2=97,0.895636,0.033868,0.829256,0.962015,0.000000e+00
id2=98,0.897023,0.034026,0.830334,0.963712,0.000000e+00


### Ultra Wide

In [15]:
N = 5_000_000
df = pd.DataFrame({ 
    'x1': np.random.rand(N), 
    'x2': np.random.rand(N), 
    'id1': np.ceil(10*np.arange(N)/N+1e-7).astype(int),
    'id2': np.random.randint(1, 100_001, size=N)
})
df['y'] = 1 + 2*df['x1'] + 3*df['x2'] + np.log10(df['id1']) + np.log10(df['id2']) + np.random.randn(N)
print(df[['id1', 'id2']].nunique())

id1        10
id2    100000
dtype: int64


In [16]:
%time fr.ols(y=R.y, x=I+R.x1+R.x2+C.id1, hdfe=C.id2, data=df)

CPU times: user 1.98 s, sys: 681 ms, total: 2.66 s
Wall time: 1.94 s


y,coeff,stderr,low95,high95,pvalue
I,0.999564,0.140022,0.725126,1.274002,9.428014e-13
x1,1.996733,0.001565,1.993666,1.999800,0.000000e+00
x2,3.000919,0.001565,2.997852,3.003987,0.000000e+00
id1=2,0.298813,0.002020,0.294854,0.302772,0.000000e+00
id1=3,0.480118,0.002020,0.476158,0.484077,0.000000e+00
...,...,...,...,...,...
id2=99996,4.908693,0.219470,4.478539,5.338848,0.000000e+00
id2=99997,5.077895,0.198005,4.689813,5.465977,0.000000e+00
id2=99998,4.908166,0.198992,4.518148,5.298184,0.000000e+00
id2=99999,5.176182,0.191175,4.801486,5.550878,0.000000e+00
